In [1]:
import tensorflow as tf
import pathlib
import imageio
#tf.enable_eager_execution()
import os
import pandas as pd


path=os.getcwd()+'\\\sp-society-camera-model-identification'
os.chdir(path)


In [2]:
print(tf.config.list_physical_devices('GPU'))
print(tf.test.gpu_device_name())

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
/device:GPU:0


In [3]:
trainpath = path+"\\train"
os.chdir(trainpath)

df1 = pd.read_pickle('train_images1.pkl')
df1.head(5)


,location,image
0,D:\Study\Capstone\cameraDetection\sp-society-c...,"(((tf.Tensor(0.47058824, shape=(), dtype=float..."
1,D:\Study\Capstone\cameraDetection\sp-society-c...,"(((tf.Tensor(0.6745098, shape=(), dtype=float3..."
2,D:\Study\Capstone\cameraDetection\sp-society-c...,"(((tf.Tensor(0.59607846, shape=(), dtype=float..."
3,D:\Study\Capstone\cameraDetection\sp-society-c...,"(((tf.Tensor(0.41568628, shape=(), dtype=float..."
4,D:\Study\Capstone\cameraDetection\sp-society-c...,"(((tf.Tensor(0.5764706, shape=(), dtype=float3..."


In [4]:
df1.shape

(1375, 2)

In [5]:
df2 = pd.read_pickle('train_images2.pkl')
df2.shape

(1375, 2)

In [6]:

df1 = pd.concat([df1,df2])
df1.shape

(2750, 2)

In [7]:
df1['camera'] = df1['location'].apply(lambda x: x.split('\\')[-2])
df1.head()    

,location,image,camera
0,D:\Study\Capstone\cameraDetection\sp-society-c...,"(((tf.Tensor(0.47058824, shape=(), dtype=float...",HTC-1-M7
1,D:\Study\Capstone\cameraDetection\sp-society-c...,"(((tf.Tensor(0.6745098, shape=(), dtype=float3...",HTC-1-M7
2,D:\Study\Capstone\cameraDetection\sp-society-c...,"(((tf.Tensor(0.59607846, shape=(), dtype=float...",HTC-1-M7
3,D:\Study\Capstone\cameraDetection\sp-society-c...,"(((tf.Tensor(0.41568628, shape=(), dtype=float...",HTC-1-M7
4,D:\Study\Capstone\cameraDetection\sp-society-c...,"(((tf.Tensor(0.5764706, shape=(), dtype=float3...",HTC-1-M7


In [8]:

cameras = os.listdir()
cameras.remove('.gitignore')
cameras.remove('train_images1.pkl')
cameras.remove('train_images2.pkl')
cameras.remove('model_weights.h5')
filecount = [len(os.listdir(camera))  for camera in cameras]
cam = pd.DataFrame({'camera_type':cameras,'filecount':filecount})

class_dict = {}
for id in cam.index:
    class_dict.update({cam.loc[id,'camera_type']:id})
    
class_dict

{'HTC-1-M7': 0,
 'iPhone-4s': 1,
 'iPhone-6': 2,
 'LG-Nexus-5x': 3,
 'Motorola-Droid-Maxx': 4,
 'Motorola-Nexus-6': 5,
 'Motorola-X': 6,
 'Samsung-Galaxy-Note3': 7,
 'Samsung-Galaxy-S4': 8,
 'Sony-NEX-7': 9}

In [9]:
df1['class'] = df1['camera'].apply(lambda x:class_dict[x])
df1.head()

,location,image,camera,class
0,D:\Study\Capstone\cameraDetection\sp-society-c...,"(((tf.Tensor(0.47058824, shape=(), dtype=float...",HTC-1-M7,0
1,D:\Study\Capstone\cameraDetection\sp-society-c...,"(((tf.Tensor(0.6745098, shape=(), dtype=float3...",HTC-1-M7,0
2,D:\Study\Capstone\cameraDetection\sp-society-c...,"(((tf.Tensor(0.59607846, shape=(), dtype=float...",HTC-1-M7,0
3,D:\Study\Capstone\cameraDetection\sp-society-c...,"(((tf.Tensor(0.41568628, shape=(), dtype=float...",HTC-1-M7,0
4,D:\Study\Capstone\cameraDetection\sp-society-c...,"(((tf.Tensor(0.5764706, shape=(), dtype=float3...",HTC-1-M7,0


In [10]:
df1['image_array'] = df1['image'].apply(lambda x:x.numpy())

In [11]:
from keras.preprocessing.image import ImageDataGenerator

Using TensorFlow backend.


In [12]:
datagen = ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True)

In [13]:
from sklearn.model_selection import train_test_split

Xtrain, Xtest, Ytrain, Ytest = train_test_split(df1['image_array'], df1['class'].values,test_size=.33, random_state=42)


In [29]:
import numpy as np
def reshape_input(Xtrain):
    #Xtrain = np.stack(Xtrain.values) #convert a pandas series of arrays into one single array
    input_cols = Xtrain.shape[1]*Xtrain.shape[2]*Xtrain.shape[3]
    Xtrain = np.reshape(Xtrain,(Xtrain.shape[0],input_cols)) #flatten the data to single set of neurons
    return Xtrain

In [15]:
import keras
from keras.utils.np_utils import to_categorical

def reshape_target(Ytrain):
    Ytrain = to_categorical(Ytrain,num_classes=10)
    return Ytrain

In [16]:
np.stack(Xtrain.values).shape

(1842, 192, 192, 3)

In [27]:
Ytrain.shape

(1842,)

In [31]:
trainingsize = 6000
batchsize = 64
iteration = 1
itersize = trainingsize/batchsize
for batch in datagen.flow(np.stack(Xtrain.values),Ytrain,batch_size= batchsize):
    batch_x = reshape_input(batch[0])
    batch_y = reshape_target(batch[1])
    print(batch_x.shape)
    break
    
    #model.fit_generator(batch,validation_split=.3,epochs=40)
    #iteration+=1
    #if(iteration >itersize )
    #    break

(64, 110592)
